In [1]:
from pandas import read_csv, DataFrame
from os import getcwd

In [2]:
##########################################################################################
def get_task_data():
    my_file_dir = getcwd() + "\\movement_time.csv"
    df = read_csv(my_file_dir)
    return df


##########################################################################################
def replace_outliers_within_group(df:DataFrame, group_columns:list, target_column:str):
    # Reset index to ensure group_columns are not in the index
    df = df.reset_index(drop=True)

    # Function to calculate and replace outliers within each group
    def replace(group):
        Q1 = group[target_column].quantile(0.25)
        Q3 = group[target_column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mean_value = group[(group[target_column] >= lower_bound) & (group[target_column] <= upper_bound)][target_column].mean()
        group[target_column] = group[target_column].apply(lambda x: mean_value if x < lower_bound or x > upper_bound else x)
        return group

    # Apply the function to each subgroup
    return df.groupby(group_columns).apply(replace).reset_index(drop=True)

In [3]:
df = get_task_data()
df.head(5)

,pid,condition,time
0,1,yes,89.317
1,1,no,52.200
2,2,yes,36.629
3,2,no,105.725
4,3,yes,509.568


In [12]:
pid_list = []
order_list = []
time_diff_list = []
time_diff_order_list = []

for part_id in range(1, 25):
    pid_list.append(part_id)
    pdf = df[df['pid']==part_id]
    time_lst = pdf['time'].tolist()
    print(time_lst)
    if part_id < 13:
        # print("part_id less")
        order_list.append("viz_first")   # yes -> no
        time_diff = time_lst[1] - time_lst[0]
        time_diff_order = time_lst[0] - time_lst[1]
        time_diff_list.append(time_diff)
        time_diff_order_list.append(time_diff_order)
    else:
        # print("part id more")
        order_list.append("none_first")    # no -> yes
        time_diff = time_lst[0] - time_lst[1]
        time_diff_order = time_lst[1] - time_lst[0]
        time_diff_list.append(time_diff)
        time_diff_order_list.append(time_diff_order)
        
print(pid_list)
print(order_list)
print(time_diff_list)

[89.317, 52.2]
[36.629, 105.725]
[509.568, 121.242]
[307.271, 188.917]
[152.579, 85.672]
[251.613, 128.886]
[212.199, 200.265]
[58.614, 106.789]
[281.966, 131.84]
[308.775, 161.278]
[113.49, 95.614]
[181.109, 163.405]
[149.237, 198.849]
[138.109, 201.697]
[86.046, 139.272]
[140.136, 481.382]
[135.532, 359.888]
[89.552, 287.084]
[99.073, 238.626]
[290.834, 541.993]
[175.328, 379.81]
[180.001, 176.573]
[133.392, 202.42]
[91.405, 102.281]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
['viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'viz_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first', 'none_first']
[-37.11699999999999, 69.096, -388.32599999999996, -118.35400000000001, -66.90700000000001, -122.727, -11.934000000000026, 48.175000000000004, -150.

In [14]:
# generate new dataframe
df_dict = {
    'pid': pid_list,
    'cond_order': order_list,
    'time_diff': time_diff_list,
    'time_diff_order': time_diff_order_list
}
td_df = DataFrame(df_dict).sort_values(by="pid")
td_df = replace_outliers_within_group(td_df, ['cond_order'], "time_diff")
td_df = replace_outliers_within_group(td_df, ['cond_order'], "time_diff_order")

# write new dataframe to csv file
dest_path = getcwd() + "\\learning.csv"
td_df.to_csv(dest_path, index=False)